In [1]:
import numpy as np
import pandas as pd
from data_processing_helper import *
import warnings
warnings.filterwarnings('ignore')
from statistics import multimode
from statistics import mode
import itertools

In [7]:
# helper functions
def get_mode(lst):
    if len(lst) == 0:
        return np.NaN
    return mode(lst)


def get_bursts(lst):
    interval = 120
    allBursts = []

    i = 0
    while i < len(lst):
        oneBurst = []
        limit = lst[i] + interval
        j = i
        while j < len(lst) and lst[j] <= limit:
            oneBurst.append(lst[j])
            j += 1
        allBursts.append(oneBurst)
        i = j
        
    return allBursts


def get_mean_num_rewards(lst):
    bursts = [i for i in lst if len(i) > 1]
    total_rewards = len(list(itertools.chain.from_iterable(bursts)))
    total_bursts = len(bursts)
    
    if total_bursts == 0:
        return 0
    
    return round(total_rewards / total_bursts,2)


def get_burst_rewards_pct(lst):
    numRewards = len(list(itertools.chain.from_iterable(lst)))
    if numRewards == 0:
        return np.NaN
    bursts = [i for i in lst if len(i) > 1]
    numBurstRewards = len(list(itertools.chain.from_iterable(bursts)))
    return round(numBurstRewards/numRewards,2) * 100 


def get_max_burst(lst):
    bursts = [i for i in lst if len(i) > 1]
    if len(bursts) == 0:
        return 0
    return max([len(i) for i in bursts])


# test
def calculations_single(df):
    # combine all reward timestamp into a column of lists
    filtered_cols = ['Subject'] + [col for col in df.columns if 'Reward ' in col]
    df_reward = df[filtered_cols].sort_values('Subject').reset_index().drop('index',axis=1)
    df_reward['allRewards'] = df_reward.iloc[:,1:].values.tolist()

    # get the filtered df
    dff = df_reward[['Subject', 'allRewards']]

    # retrieve the inter-reward intervals, filtering out negatives
    dff['Intervals'] = dff['allRewards'].apply(lambda lst:[j-i for i, j in zip(lst[:-1], lst[1:])])
    dff['cleanedIntervals'] = dff['Intervals'].apply(lambda lst: [val for val in lst if val > 0])

    # calculate needed traits related to the inter-reward intervals
    dff['meanInterval'] = dff['cleanedIntervals'].apply(lambda x: round(np.mean(x), 2))
    dff['stdInterval'] = dff['cleanedIntervals'].apply(lambda x: round(np.std(x), 2))
    dff['modeInterval'] = dff['cleanedIntervals'].apply(get_mode)

    # filtering the rewards (zero values)
    dff['cleanedRewards'] = dff['allRewards'].apply(lambda lst: [val for val in lst if val > 0])
    dff['totalRewards'] = dff['cleanedRewards'].apply(lambda x: len(x))

    # retrieve the "bursts" (cluster of rewards happened within 2 mins) from rewards 
    dff['rawBurst'] = dff['cleanedRewards'].apply(get_bursts)
    dff['numBurst'] = dff['rawBurst'].apply(lambda x: len([i for i in x if len(i)>1]))

    # get the mean number of rewards across all the bursts
    dff['meanNumRewards'] = dff['rawBurst'].apply(get_mean_num_rewards)

    # get the percentage of rewards that fall in burst out of all the rewards
    dff['pctRewards'] = dff['rawBurst'].apply(get_burst_rewards_pct)

    # get the maximum number of rewards contain in a single burst in one session
    dff['maxBurst'] = dff['rawBurst'].apply(get_max_burst)

    # select needed columns for output df
    output_cols = ['Subject', 'meanInterval', 'stdInterval', 'modeInterval','meanNumRewards', 'numBurst', 'maxBurst', 'pctRewards']
    dff_out = dff[output_cols]
    
    return dff_out

In [8]:
filepath = './SHA/BSB273BC04HSOXYSHA01_output.xlsx'
df = cleanup(filepath)

In [9]:
calculations_single(df)

,Subject,meanInterval,stdInterval,modeInterval,meanNumRewards,numBurst,maxBurst,pctRewards
0,F401,755.86,520.27,1356.0,2.0,2,2,50.0
1,F402,1730.25,2211.53,1327.0,2.0,1,2,40.0
2,F403,NaN,NaN,NaN,0.0,0,0,0.0
3,F404,519.00,354.00,165.0,0.0,0,0,0.0
4,F405,591.50,396.56,27.0,2.0,1,2,22.0
5,F406,5228.00,0.00,5228.0,0.0,0,0,0.0
6,F407,NaN,NaN,NaN,0.0,0,0,NaN
7,F408,NaN,NaN,NaN,0.0,0,0,0.0
8,F409,1128.75,1055.93,27.0,2.0,1,2,40.0
9,F410,NaN,NaN,NaN,0.0,0,0,NaN
